In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/Othercomputers/HomePC/Neurophysiological-Data-Decoding/ 

!pip install git+https://github.com/cvnlab/GLMsingle.git
!pip install nilearn pybids

/content/drive/Othercomputers/HomePC/Neurophysiological-Data-Decoding
  Cloning https://github.com/cvnlab/GLMsingle.git to /tmp/pip-req-build-q3_fn7wi
  Running command git clone -q https://github.com/cvnlab/GLMsingle.git /tmp/pip-req-build-q3_fn7wi
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 6.8 MB 13.7 MB/s 
  Created wheel for GLMsingle: filename=GLMsingle-0.0.1-py3-none-any.whl size=99948 sha256=2b1a03d35f11da90302d6619df264ccb5edadc7d202ca3a1bd852b23d5400f1e
  Stored in directory: /tmp/pip-ephem-wheel-cache-sqvb_wtr/wheels/ce/41/bf/3cef35a0ec418c547c8e46d7794a95f660fc371bd514345cc6
Successfully built GLMsingle
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the f

In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pprint import pprint
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from tqdm.notebook import tqdm
import nibabel as nib
import glmsingle
from glmsingle.glmsingle import GLM_single

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

from research.experiments.tc2see.analysis import (
    get_design_matrix,
)
from research.experiments.tc2see.data_loading import (
    Data,
)

In [4]:
subject = "01"  # subjects 01 and 02
task = "bird"
space = "T1w"
#space = "MNI152NLin2009cAsym"
tr = 1.97
version = 1
dataset_path = Path(f"/content/drive/Othercomputers/HomePC/TC2See_v{version}")
num_runs = 6 if version == 1 else 8

#data = Data(str(dataset_path), subject, task, 1, space)
#data.events

In [5]:
from scipy.ndimage import zoom

# Load runs
resample_tr = 0.2
fmri_mask = None
fmri_batch = []
events_batch = []
for run in tqdm(range(1, num_runs + 1)):
    data = Data(str(dataset_path), subject, task, run, space)
    events_batch.append(data.events)

    fmri_data = data.fmri_img.get_fdata()
    if fmri_mask is None:
        fmri_mask = data.mask.get_fdata().astype(bool)
    fmri_data_masked = fmri_data[fmri_mask]
    fmri_data_masked = zoom(fmri_data_masked, zoom=(1, tr / resample_tr), grid_mode=False)
    fmri_batch.append(fmri_data_masked)

  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/bids/config.py:40: FutureWarning: Setting 'extension_initial_dot' will be removed in pybids 0.16.
  FutureWarning)


fmri_img /content/drive/Othercomputers/HomePC/TC2See_v1/derivatives_TC2See_prdgm/fmriprep/sub-01/func/sub-01_task-bird_run-1_space-T1w_desc-preproc_bold.nii.gz
mask /content/drive/Othercomputers/HomePC/TC2See_v1/derivatives_TC2See_prdgm/fmriprep/sub-01/func/sub-01_task-bird_run-1_space-T1w_desc-brain_mask.nii.gz
fmri_img /content/drive/Othercomputers/HomePC/TC2See_v1/derivatives_TC2See_prdgm/fmriprep/sub-01/func/sub-01_task-bird_run-2_space-T1w_desc-preproc_bold.nii.gz
fmri_img /content/drive/Othercomputers/HomePC/TC2See_v1/derivatives_TC2See_prdgm/fmriprep/sub-01/func/sub-01_task-bird_run-3_space-T1w_desc-preproc_bold.nii.gz
fmri_img /content/drive/Othercomputers/HomePC/TC2See_v1/derivatives_TC2See_prdgm/fmriprep/sub-01/func/sub-01_task-bird_run-4_space-T1w_desc-preproc_bold.nii.gz
fmri_img /content/drive/Othercomputers/HomePC/TC2See_v1/derivatives_TC2See_prdgm/fmriprep/sub-01/func/sub-01_task-bird_run-5_space-T1w_desc-preproc_bold.nii.gz
fmri_img /content/drive/Othercomputers/HomePC/

In [6]:
# Collect all conditions
conditions = []
for run_id, events in enumerate(events_batch):
    for id, event in events.iterrows():
        if not event['stimulus'].endswith('png'):
            continue
        condition_name = Path(event['stimulus']).stem.split('.')[1]
        conditions.append(condition_name)
conditions = list(set(conditions))
conditions.sort()
conditions = {condition: i  for i, condition in enumerate(conditions)}
C = len(conditions)

_, T = fmri_batch[0].shape

design_batch = []
for run_id, events in enumerate(events_batch):
    _, T = fmri_batch[run_id].shape
    design_matrix = np.zeros(shape=(T, C))
    for id, event in events.iterrows():
        if not event['stimulus'].endswith('png'):
            continue
        condition_name = Path(event['stimulus']).stem.split('.')[1]
        c = conditions[condition_name]
        t = round(event.tr / resample_tr)
        design_matrix[t, c] = 1
    design_batch.append(design_matrix)

@interact(run_id=(0, len(design_batch)-1))
def show(run_id):
    design_matrix = design_batch[run_id]
    plt.figure(figsize=(12, 12))
    plt.imshow(design_matrix)

interactive(children=(IntSlider(value=2, description='run_id', max=5), Output()), _dom_classes=('widget-intera…

In [ ]:
glmsingle_obj = GLM_single(dict(
    wantlibrary=1,
    wantglmdenoise=1,
    wantfracridge=1,
    wantfileoutputs=[1,1,1,1],
    wantmemoryoutputs=[1,1,1,1],
))

pprint(glmsingle_obj.params)

output_path = dataset_path / f'derivatives_TC2See_prdgm/glmsingle_{int(resample_tr * 1000)}ms'
results_glmsingle = glmsingle_obj.fit(
    design=design_batch,
    data=fmri_batch,
    stimdur=2,
    tr=data.tr,
    outputdir=str(output_path),
)

{'R2thresh': 0,
 'brainR2': [],
 'brainexclude': False,
 'brainthresh': [99.0, 0.1],
 'chunklen': 50000,
 'extra_regressors': False,
 'fracs': array([1.  , 0.95, 0.9 , 0.85, 0.8 , 0.75, 0.7 , 0.65, 0.6 , 0.55, 0.5 ,
       0.45, 0.4 , 0.35, 0.3 , 0.25, 0.2 , 0.15, 0.1 , 0.05]),
 'hrffitmask': 1,
 'hrfmodel': 'optimise',
 'hrfthresh': 0.5,
 'lambda': 0,
 'n_boots': 100,
 'n_jobs': 1,
 'n_pcs': 10,
 'numforhrf': 50,
 'pcR2cutoff': [],
 'pcR2cutoffmask': 1,
 'pcstop': 1.05,
 'seed': 1651175766.7010949,
 'suppressoutput': 0,
 'wantautoscale': 1,
 'wantfileoutputs': [1, 1, 1, 1],
 'wantfracridge': 1,
 'wantglmdenoise': 1,
 'wanthdf5': 0,
 'wantlibrary': 1,
 'wantlss': 0,
 'wantmemoryoutputs': [1, 1, 1, 1],
 'wantparametric': 0,
 'wantpercentbold': 1}
*** FITTING TYPE-A MODEL (ONOFF) ***



In [ ]:
type_names = {
    'typea': 'onoff',
    'typeb': 'fithrf',
    'typec': 'fithrf_GLMdenoise',
    'typed': 'fithrf_GLMdenoise_RR'
}

num_voxels = fmri_mask.sum()
for type_name, results in results_glmsingle.items():
    result_name = type_names[type_name]
    print(result_name)
    result_path = output_path / result_name
    result_path.mkdir(exist_ok=True, parents=True)

    for k, v in results.items():
        if isinstance(v, np.ndarray):
            print(k, v.dtype, v.shape)
            if v.shape[0] == num_voxels:
                if len(v.shape) == 1:
                    out_img = np.zeros_like(fmri_mask, dtype=v.dtype if v.dtype != bool else int)
                    out_img[fmri_mask] = v
                    img = nib.Nifti1Image(out_img, np.eye(4))
                    nib.save(img, result_path / f'{result_name}_{k}.nii.gz')

        else:
            print(k, type(v))

onoff
onoffR2 float64 (149157,)
meanvol float32 (149157,)
betasmd float32 (149157, 1)
fithrf
FitHRFR2 float32 (149157, 20)
FitHRFR2run float32 (894942, 20)
HRFindex int64 (149157,)
HRFindexrun int64 (894942,)
R2 float32 (149157,)
R2run float32 (149157, 6)
betasmd float32 (149157, 672)
meanvol float32 (149157,)
fithrf_GLMdenoise
HRFindex int64 (149157,)
HRFindexrun int64 (894942,)
glmbadness float32 (149157, 11)
pcvoxels bool (149157,)
pcnum <class 'int'>
xvaltrend float32 (11,)
noisepool bool (149157,)
pcregressors <class 'list'>
betasmd float32 (149157, 672)
R2 float32 (149157,)
R2run float32 (149157, 6)
meanvol float32 (149157,)
fithrf_GLMdenoise_RR
HRFindex int64 (149157,)
HRFindexrun int64 (894942,)
glmbadness float32 (149157, 11)
pcvoxels bool (149157,)
pcnum <class 'int'>
xvaltrend float32 (11,)
noisepool bool (149157,)
pcregressors <class 'list'>
betasmd float32 (149157, 672)
R2 float32 (149157,)
R2run float32 (149157, 6)
rrbadness float32 (149157, 20)
FRACvalue float32 (149157,

In [ ]:
fmri_mask.sum()

169893